Data Preparation

In [1]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import scale

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
plt.rcParams['font.family'] = ['Microsoft JhengHei']  # 設定中文字體為微軟正黑
plt.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings("ignore")
primaryColor = '#fd4d58'
secondaryColor = '#969494'
sns.set(style='white',font ='Microsoft JhengHei', font_scale=2)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_excel('/content/drive/MyDrive/112-1 MDS Final Project/Dataset/1126 Preprocessed Data.xlsx')

In [5]:
df.head()

,host_acceptance_rate,host_is_superhost,host_has_profile_pic,accommodates,bedrooms,beds,price,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,host_since_days,last_review_days,shared_bath,private_bath,bath,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within an hour,room_type_Hotel room,room_type_Private room,room_type_Shared room,neighbourhood_cleansed_中山區,neighbourhood_cleansed_中正區,neighbourhood_cleansed_信義區,neighbourhood_cleansed_內湖區,neighbourhood_cleansed_北投區,neighbourhood_cleansed_南港區,neighbourhood_cleansed_士林區,neighbourhood_cleansed_大同區,neighbourhood_cleansed_大安區,neighbourhood_cleansed_文山區,neighbourhood_cleansed_松山區,neighbourhood_cleansed_萬華區,property_type_Barn,property_type_Cave,property_type_Entire bungalow,property_type_Entire chalet,property_type_Entire condo,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Farm stay,property_type_Minsu,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in bungalow,property_type_Private room in casa particular,property_type_Private room in cave,property_type_Private room in condo,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in home,property_type_Private room in hostel,property_type_Private room in loft,property_type_Private room in minsu,property_type_Private room in rental unit,property_type_Private room in serviced apartment,property_type_Private room in tent,property_type_Private room in tiny home,property_type_Private room in townhouse,property_type_Private room in villa,property_type_Room in aparthotel,property_type_Room in boutique hotel,property_type_Room in heritage hotel,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room,property_type_Shared room in aparthotel,property_type_Shared room in bed and breakfast,property_type_Shared room in boutique hotel,property_type_Shared room in cave,property_type_Shared room in condo,property_type_Shared room in guest suite,property_type_Shared room in home,property_type_Shared room in hostel,property_type_Shared room in loft,property_type_Shared room in minsu,property_type_Shared room in rental unit,property_type_Shared room in serviced apartment,property_type_Tiny home,air conditioning,baby stuffs,Backyard,balcony,BBQ,beach,bedroom supplies,bidet,books,breakfast,building staff,cable,changing table,children stuffs,cleaning available during stay,cleaning products,clothing storage,clothing supplies,coffee,ceiling fan,dining table,dreeser,dryer,elevator,essentials,ethernet connection,EV charger,exercise equipment,fire extinguisher,first aid kit,game,gas,gym,hammock,heating,host greets you,hot water,infinity,Iron,keypad,kitchen,lake access,laundromat nearby,lock,long term stays allowed,luggage dropoff allowed,microwave,mosquito net,open 24 hours,open specific hours,outdoor dining area,outdoor furniture,outdoor shower,outlet covers,oven,parking,pets allowed,piano,pool,portable fans,private entrance,private living room,record player,refrigerator,resort access,rooftop,room-darkening shades,safe,sauna,security cameras on property,self check-in,single level home,smoke alarm,smoking allowed,sound system,stove,streaming service,streaming services,table corner guards,trash compactor,tub,TV,utensils,view,washer,waterfront,wifi,window guards,workspace
0,83.0,1,1,2,1.0,1.0,1029,30.0,365.0,1,19,294,7,0,0,5.00,5.00,4.71,0,0.05,4644,1638.0,0.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

In [6]:
features_enet = pd.read_excel('/content/drive/MyDrive/112-1 MDS Final Project/Dataset/1202 Elastic_Net_Feature.xlsx')
features_voting4 = pd.read_excel('/content/drive/MyDrive/112-1 MDS Final Project/Dataset/1202 Voting_4_Feature.xlsx')
features_voting4plus3 = pd.read_excel('/content/drive/MyDrive/112-1 MDS Final Project/Dataset/1202 Voting_3&4_Feature.xlsx')

In [7]:
target = pd.DataFrame([{'feature':'price'}])
features_enet = pd.concat([features_enet, target],ignore_index=True)
features_voting4 = pd.concat([features_voting4, target],ignore_index=True)
features_voting4plus3 = pd.concat([features_voting4plus3, target],ignore_index=True)

In [8]:
# 來篩選 df 中的欄位
enet_df = df[features_enet['feature'].values]
voting4_df = df[features_voting4['feature'].values]
voting4plus3_df = df[features_voting4plus3['feature'].values]

In [9]:
# feature scaling
# enet_df = pd.DataFrame(scale(enet_df),columns=enet_df.columns)
# voting4_df = pd.DataFrame(scale(voting4_df),columns=voting4_df.columns)
# voting4plus3_df = pd.DataFrame(scale(voting4plus3_df),columns=voting4plus3_df.columns)

Linear Regression

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

In [11]:
def linear_reg(dataset):
  y = dataset[['price']]
  X = dataset.drop(columns=['price'])
  # X = pd.DataFrame(scale(X),columns=X.columns)

  X_columns = X.columns
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  print(len(X_train), 'rows in training set')
  print(len(X_test), 'rows in test set')

  clf = LinearRegression()

  #候選參數
  parameters = {
      'n_jobs': [1, 2, 5, 10, 100],
      'fit_intercept': [True, False]
  }
  # 網格搜索交叉驗證來調參數
  cv = GridSearchCV(estimator=clf, param_grid=parameters, cv=5, verbose=3)
  cv.fit(X_train,y_train)
  # 預測testing data
  pred = cv.predict(X_test)
  #模型評估
  r2 = r2_score(y_test, pred)
  Adj_r2 = 1 - (1-r2) * (len(y_test)-1)/(len(y_test)-X.shape[1]-1)
  mse = mean_squared_error(y_test, pred)
  rmse = mse **.5

  # 最佳参数
  best_par = cv.best_params_
  print("The best parameters: {}".format(best_par))
  coefficients = cv.best_estimator_.coef_
  #特徵重要度
  importance = np.abs(coefficients)
  feature_importance = pd.DataFrame(importance, columns=X_columns).T
  #feature_importance = feature_importance.T
  feature_importance.columns = ['importance']
  feature_importance = feature_importance.sort_values('importance', ascending=False)
  print("The model performance for testing set")
  print("--------------------------------------")
  print('RMSE is {}'.format(rmse))
  print('R2 score is {}'.format(r2))
  print("\n")

  return feature_importance, rmse, r2, Adj_r2


In [12]:
df_list = [enet_df, voting4_df, voting4plus3_df]
for dataset in df_list:
  feature_importance, rmse, r2, Adj_r2 = linear_reg(dataset)

  print(feature_importance)
  print("\n")
  print('RMSE is {}'.format(rmse))
  print("\n")
  print('R2 score is {}'.format(r2))
  print("\n")
  print('Adj_R2 score is {}'.format(Adj_r2))
  print("\n")

3777 rows in training set
945 rows in test set
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ......fit_intercept=True, n_jobs=1;, score=0.495 total time=   0.1s
[CV 2/5] END ......fit_intercept=True, n_jobs=1;, score=0.500 total time=   0.1s
[CV 3/5] END ......fit_intercept=True, n_jobs=1;, score=0.512 total time=   0.1s
[CV 4/5] END ......fit_intercept=True, n_jobs=1;, score=0.506 total time=   0.1s
[CV 5/5] END ......fit_intercept=True, n_jobs=1;, score=0.573 total time=   0.1s
[CV 1/5] END ......fit_intercept=True, n_jobs=2;, score=0.495 total time=   0.1s
[CV 2/5] END ......fit_intercept=True, n_jobs=2;, score=0.500 total time=   0.1s
[CV 3/5] END ......fit_intercept=True, n_jobs=2;, score=0.512 total time=   0.1s
[CV 4/5] END ......fit_intercept=True, n_jobs=2;, score=0.506 total time=   0.1s
[CV 5/5] END ......fit_intercept=True, n_jobs=2;, score=0.573 total time=   0.1s
[CV 1/5] END ......fit_intercept=True, n_jobs=5;, score=0.495 total time=   0.1s
[